<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Group_Number: 124
#### Student Name: Pankaj Shitole
#### Student ID: 33570523
#### Student Name: Sachin Shivaramaiah
#### Student ID: 34194037


Date: 30/08/2024


Environment: google Colab

---




Libraries used:
* re (for regular expression, installed and imported)
* pandas (for data manipulation)
* datetime
* json
    
</div>

## 1. **INTRODUCTION**
Task 1 focuses on the essential first step of processing raw text data by extracting and structuring information from semi-structured Google Maps reviews. The task involves parsing various data formats, including mis-structured XML files and Excel sheets, to transform unorganized data into a structured format. The goal is to produce CSV and JSON files that summarize key metrics, such as review counts, text presence, and response times. This structured output will serve as the foundation for further analysis in subsequent tasks.

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

os: Helps in managing file operations, such as navigating directories and handling file paths, ensuring smooth access and organization of the input and output files.

pandas: Provides powerful tools for data manipulation and transformation, allowing us to efficiently process, clean, and analyze structured data, ultimately producing the required CSV outputs.

re: Enables pattern matching and text extraction from the semi-structured text files, which is essential for accurately parsing and processing raw review data.

datetime: Assists in handling and formatting date and time data, allowing us to standardize review timestamps and calculate metrics like the earliest and latest review dates.

json: Facilitates the creation and management of JSON outputs, ensuring that the extracted data is structured correctly for further analysis and storage




In [1]:
# Import required libraries
import os
import pandas as pd
import re
from datetime import datetime
import json

Mount the drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Folder path to access the files

In [3]:
folder_path = '/content/drive/Shareddrives/FIT5196_S2_2024/GroupAssessment1/Student Data/student_group124'

Change the current working directory

In [4]:
os.chdir(folder_path)

<div class="alert alert-block alert-warning">
    
## 3.  Loading and Reading Files <a class="anchor" name="libs"></a>
 </div>

**INCEPTION STEP:** The inception point would be to start from reading the file from the designated folder path which will be done with utf-8 encoding

In [5]:
def read_txt_files(file_path):
    with open(f'{folder_path}/{file_path}', 'r', encoding = 'utf-8') as f:    # Passed folder path and file path
        return f.read()

Now iterate through the list of files in the current directory and store the string values of the text file in a list

In [6]:
text_files = []
excel_file = []

for file in os.listdir():    # list all files in the directory
    if file.endswith('.txt'):    # Check the file which is ending with .txt
        text_files.append(read_txt_files(file))
    else:
      excel_dictionary = pd.read_excel(file, engine = 'openpyxl', sheet_name = None)    # Reading multiple sheets in an excel file. Reference [1]
      # By default it is ready in dict() datatype
      # Converting it into list, since it's easy to parse and we are not required to store sheet names
      excel_file = list(excel_dictionary.values())

Check for number of files

In [7]:
len(text_files), len(excel_file)

(15, 16)

<div class="alert alert-block alert-warning">
    
## 4. Regex-Based Pattern Matching and Storage  <a class="anchor" name="libs"></a>
 </div>

Storing all the patterns in the variables

In [8]:
# Using (?i) to ignore the case. Reference provided
userid_pattern = r"(?i)<\s*user[_id.\s]*>\s*(.*?)\s*<\s*[/]*\s*user[_id.\s]*>"
gmapid_pattern = r"(?i)<\s*gmap[_\s]*id\s*>\s*(.*?)\s*<\s*/{1,2}\s*gmap[_\s]*id\s*>"
pics_pattern = r"(?i)<\s*(?:pics|pictures)\s*>\s*(.*?)\s*<\s*/{1,2}\s*(?:pics|pictures)\s*>"
review_pattern = r"(?i)<\s*(?:text|review)\s*>\s*(.*?)\s*<\s*/{1,2}\s*(?:text|review)\s*>"
rating_pattern = r"(?i)<\s*(?:rate|rating)\s*>\s*(.*?)\s*<\s*/{1,2}\s*(?:rate|rating)\s*>"
time_pattern = r"(?i)<\s*(?:time|date)\s*>\s*(.*?)\s*<\s*/{1,2}\s*(?:time|date)\s*>"
resp_pattern = r"(?i)<\s*(?:resp|response)\s*>\s*(.*?)\s*<\s*/{1,2}\s*(?:resp|response)\s*>"

In [9]:
patterns_list = [userid_pattern, resp_pattern, pics_pattern, review_pattern, rating_pattern, time_pattern, gmapid_pattern]

In [10]:
matches_count = []
for text in text_files:
  lst = []
  for patterns in patterns_list:
    matches = re.findall(patterns, text, re.DOTALL|re.IGNORECASE)    # Added the flags to ignore the case and to access the new line character using .*. Reference [2]
    lst.append(matches)
  matches_count.append(lst)


In [11]:
len(matches_count[0][0])

4488

Check the number of tags extracted per file. It will give us an idea about consistency in fetching the tags

In [12]:
for number, file_ in enumerate(matches_count):
  print(f"File number: {number}")
  for tag in file_:
    print(len(tag))

File number: 0
4488
4488
4488
4488
4488
4488
4488
File number: 1
4007
4007
4007
4007
4007
4007
4007
File number: 2
2057
2057
2057
2057
2057
2057
2057
File number: 3
3638
3638
3638
3638
3638
3638
3638
File number: 4
3292
3292
3292
3292
3292
3292
3292
File number: 5
1737
1737
1737
1737
1737
1737
1737
File number: 6
1791
1791
1791
1791
1791
1791
1791
File number: 7
2045
2045
2045
2045
2045
2045
2045
File number: 8
1988
1988
1988
1988
1988
1988
1988
File number: 9
2316
2316
2316
2316
2316
2316
2316
File number: 10
1963
1963
1963
1963
1963
1963
1963
File number: 11
2036
2036
2036
2036
2036
2036
2036
File number: 12
2223
2223
2223
2223
2223
2223
2223
File number: 13
1996
1996
1996
1996
1996
1996
1996
File number: 14
1615
1615
1615
1615
1615
1615
1615


In [13]:
# Initialize empty lists to store all matches for each pattern
all_user_ids = []
all_responses = []
all_pictures = []
all_reviews = []
all_ratings = []
all_times = []
all_gmap_ids = []

# List to store references to the lists above for easier management
all_matches_lists = [all_user_ids, all_responses, all_pictures, all_reviews, all_ratings, all_times, all_gmap_ids]

# Iterate over each match list (for each text file)
for match_lst in matches_count:
    for i, matches in enumerate(match_lst):
        # Extend the corresponding list with the matches found in this file
        all_matches_lists[i].extend(matches)


In [14]:
# Combine the data into a structure suitable for DataFrame
data = list(zip(*all_matches_lists))

# Create DataFrame with appropriate column names
df_text = pd.DataFrame(data, columns=['user_id', 'response', 'pic', 'reviews', 'ratings', 'time', 'gmapid'])


In [15]:
df_text

,user_id,response,pic,reviews,ratings,time,gmapid
0,116216747380592226401,"{'time': 1621526279693, 'text': 'Thank you for...",None,Customer Service was excellent. The Manager Ma...,5,1621385173917,0x80dd2cc88b994a75:0x28df15f7f68be2ef
1,112815741091726584474,"{'time': 1620056896251, 'text': 'Happy to help...",None,This is a welcoming place for those needing a ...,5,1619889467518,0x80dd2cc88b994a75:0x28df15f7f68be2ef
2,111318063126412318716,None,None,My belongings have been safe and secure here f...,5,1622740824982,0x80dd2cc88b994a75:0x28df15f7f68be2ef
3,107516460903115650728,"{'time': 1617555155506, 'text': 'Thank you Dan...",None,Service here has to be top tier. Martin went a...,5,1617092235345,0x80dd2cc88b994a75:0x28df15f7f68be2ef
4,105129978173299816256,"{'time': 1619468116308, 'text': 'Thank you for...",None,I had an awesome experience here! Rob was so h...,5,1619211513751,0x80dd2cc88b994a75:0x28df15f7f68be2ef
...,...,...,...,...,...,...,...
37187,109527903977692502976,None,None,None,5,1598637283598,0x808fa33b072bace5:0xd3341eec80d40259
37188,116470109907099438382,"{'time': 1617988455775, 'text': 'Thank you Mil...",None,None,5,1606846404518,0x808fa33b072bace5:0xd3341eec80d40259
37189,104872366756536408248,None,None,None,5,1578431741051,0x808fa33b072bace5:0xd3341eec80d40259
37190,106824303708774603559,"{'time': 1617986704266, 'text': 'Thank you Pau...",None,None,5,1611250935401,0x808fa33b072bace5:0xd3341eec80d40259


<div class="alert alert-block alert-warning">
    
## 5. Excel Data Parsing and Extraction  <a class="anchor" name="libs"></a>
 </div>

In [16]:
#extracting the excel_data
col_names = ['user_id', 'resp', 'pics', 'text', 'rating', 'time', 'gmap_id']
filtered_sheets = [df[col_names] for df in excel_file]


In [17]:
#Concating all the filtered sheets into one sheet
df_excel = pd.concat(filtered_sheets, ignore_index=False)

In [18]:
#Printing the excel
df_excel

,user_id,resp,pics,text,rating,time,gmap_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101793931959565944658,NaN,NaN,"Gosh, can you like a place just because of how...",5.0,1.612337e+12,0x809ade2768d0183d:0xf69bd2ae7fe733a8
4,104775924903315303964,"{'time': 1529011712830, 'text': 'Ricky Carmona...",[{'url': ['https://lh5.googleusercontent.com/p...,Great sandwiches and great service. Recommend!,5.0,1.528669e+12,0x809ade2768d0183d:0xf69bd2ae7fe733a8
...,...,...,...,...,...,...,...
189,110724902263798115155,NaN,NaN,NaN,3.0,1.475952e+12,0x80dcd7c31e793301:0x50dd0e8b33f0efbf
190,102058335035542833544,NaN,NaN,NaN,3.0,1.551473e+12,0x80dcd7c31e793301:0x50dd0e8b33f0efbf
191,109844447818377140371,"{'time': 1568820006700, 'text': ""Hi John, Than...",NaN,NaN,3.0,1.568599e+12,0x80dcd7c31e793301:0x50dd0e8b33f0efbf
192,115707407858529519575,"{'time': 1555340663137, 'text': 'Thanks, Ashle...",NaN,NaN,5.0,1.555319e+12,0x80dcd7c31e793301:0x50dd0e8b33f0efbf


In [19]:
#Preprocessing the mereged data
df_excel.dropna(axis=0, how="all", inplace=True)    # drop the nas if any

In [20]:
 # Rename the columns to make it consistent with the text files
df_excel.rename(columns={"resp": "response", "pics": "pic", "text": "reviews", "rating": "ratings", "gmap_id": "gmapid"}, inplace=True)

<div class="alert alert-block alert-warning">
    
## 6. Text and Excel Data Integration  <a class="anchor" name="libs"></a>
 </div>

In [21]:
# Concatenate the text and excel dataframe
df = pd.concat([df_excel, df_text], ignore_index= True)

In [22]:
#Print after merging
df

,user_id,response,pic,reviews,ratings,time,gmapid
0,101793931959565944658,NaN,NaN,"Gosh, can you like a place just because of how...",5.0,1612336941686.0,0x809ade2768d0183d:0xf69bd2ae7fe733a8
1,104775924903315303964,"{'time': 1529011712830, 'text': 'Ricky Carmona...",[{'url': ['https://lh5.googleusercontent.com/p...,Great sandwiches and great service. Recommend!,5.0,1528668959545.0,0x809ade2768d0183d:0xf69bd2ae7fe733a8
2,104008070789102455926,NaN,NaN,I ordered pepper jack melt with pastrami. The ...,1.0,1615853617039.0,0x809ade2768d0183d:0xf69bd2ae7fe733a8
3,117302051837968062902,NaN,NaN,Kayla is a girl that makes my sandwiches but s...,5.0,1617331195326.0,0x809ade2768d0183d:0xf69bd2ae7fe733a8
4,101898624484349413298,NaN,NaN,Love for all the sammies. Really appreciate t...,5.0,1619312448176.0,0x809ade2768d0183d:0xf69bd2ae7fe733a8
...,...,...,...,...,...,...,...
40003,109527903977692502976,None,None,None,5,1598637283598,0x808fa33b072bace5:0xd3341eec80d40259
40004,116470109907099438382,"{'time': 1617988455775, 'text': 'Thank you Mil...",None,None,5,1606846404518,0x808fa33b072bace5:0xd3341eec80d40259
40005,104872366756536408248,None,None,None,5,1578431741051,0x808fa33b072bace5:0xd3341eec80d40259
40006,106824303708774603559,"{'time': 1617986704266, 'text': 'Thank you Pau...",None,None,5,1611250935401,0x808fa33b072bace5:0xd3341eec80d40259


<div class="alert alert-block alert-warning">
    
## 7. Data Pre-processing and Tranformation  <a class="anchor" name="libs"></a>
 </div>

In [23]:
# Drop the duplicates if any
df.drop_duplicates(inplace=True)

In [24]:
# Check for response column for nan
df['response'].isna().sum()

2208

In [25]:
df['response'].fillna("None", inplace = True)

In [26]:
# Convert the 'time' column to numeric
df['time'] = pd.to_numeric(df['time'], errors='coerce')

Create a function to convert timestamp to utc

In [27]:
def convert_timestamp_to_utc(timestamp_ms):
    # Convert milliseconds to seconds
    timestamp_s = timestamp_ms / 1000.0
    # Convert to datetime object in UTC
    dt_object = datetime.utcfromtimestamp(timestamp_s)
    # Format the datetime object to the desired format
    return dt_object.strftime('%Y-%m-%d %H:%M:%S')

In [28]:
df['time'] = df['time'].apply(convert_timestamp_to_utc)    # transform the time column in utc format

In [29]:
df['time'][0]

'2021-02-03 07:22:21'

In [30]:
#Imputing None inplace of nan values
df["reviews"].fillna("None", inplace = True)

Extracting Eglish review translated by google. Steps below



In [31]:
#Checking for the structure of the reviews specially for the translated to English ones.
indices = []
for i in range(len(df)):
     if "(Translated by Google)" in df.iloc[i, 3]:
         indices.append(i)
df.iloc[indices, 3]


,reviews
323,(Translated by Google) Starbucks very cozy and...
324,(Translated by Google) Good attention. Very go...
325,(Translated by Google) Sluggish and incredibly...
326,(Translated by Google) It's excellent\n\n(Orig...
327,"(Translated by Google) Fast and standard, the ..."
...,...
37882,(Translated by Google) All organic\n\n(Origina...
37883,(Translated by Google) Perfect 👌 👍👍👍😀😀😀😀\n\n(O...
37884,(Translated by Google) Great TJs!\n\n(Original...
39135,(Translated by Google) Friendly\n\n(Original)\...


In [32]:
#Extracting the review_text which were tranlated to English by google
def extract_english_reviews(reviews):
    if "(Translated by Google)" in reviews:
        pattern = r"\(Translated by Google\)\s*(.*?)\s*\(Original\)"
        matches = re.findall(pattern, reviews, re.DOTALL)
    else:
        return reviews
    return matches[0]

In [33]:
#Extraction
df.loc[:, 'reviews'] = df['reviews'].apply(lambda x: extract_english_reviews(x))

Extract the dimensions for the pic column

In [34]:
# Check for nan values in the column
df['pic'].isna().sum()

2534

In [35]:
# Create a function to extract the pic dimension
dimension_pattern = re.compile(r'w(\d+)-h(\d+)')

# Function to extract dimensions from the 'pic' column
def extract_dimensions(pics_column):
    if pd.isna(pics_column) or pics_column in ["None", ""]:    # Considering all the None, nan and empty string if any
        return []
    else:
        # Find all matches of the pattern in the string
        dimensions = dimension_pattern.findall(pics_column)
        # Convert matches to list of lists (height, width)
        return [[str(h), str(w)] for w, h in dimensions]

In [36]:
# Apply the above function to the pic column and create a new dimensions column
df['dimensions'] = df['pic'].apply(extract_dimensions)

In [37]:
# Reorder the columns to place dimensions next to pics just for convenience
cols = df.columns.tolist()
pic_index = cols.index('pic')
cols.insert(pic_index + 1, cols.pop(cols.index('dimensions')))
df = df[cols]

<div class="alert alert-block alert-warning">
    
## 8. Data Serialization to JSON and CSV  <a class="anchor" name="libs"></a>
 </div>

Conversion to CSV file considering respective counts of the columns

In [38]:
 # create a new column to keep track of review text count
df.loc[:,'review_text_count'] = df['reviews'].apply(lambda x: 1 if x != "None" else 0)

<ipython-input-38-547bd91ccb82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'review_text_count'] = df['reviews'].apply(lambda x: 1 if x != "None" else 0)


In [39]:
#Response Count
df['response_count'] = df['response'].apply(lambda x: 1 if x != "None" else 0)

<ipython-input-39-ed06e3019e4f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['response_count'] = df['response'].apply(lambda x: 1 if x != "None" else 0)


In [40]:
# Group by gmapid and aggregate the counts
result_df = df.groupby('gmapid').agg(
    review_count=('user_id', 'count'),  # Count the number of reviews
    review_text_count=('review_text_count', 'sum'),  # Sum the text review counts
    response_count=('response_count', 'sum')  # Sum the response counts
).reset_index()

In [41]:
# Rename the gmapid column to gmap_id for consistency with output format
result_df.rename(columns={'gmapid': 'gmap_id'}, inplace=True)

In [42]:
# Print the result_df
result_df

,gmap_id,review_count,review_text_count,response_count
0,0x54cb9796e2112817:0x597cf5d29a1b0c62,107,67,0
1,0x54ce45d613e15fe5:0x44119609f38d89a1,188,178,0
2,0x54d15700d6d49a27:0xef211ab749236af7,75,51,0
3,0x54d292ba7a1c0505:0xb96e42734aa895e1,58,28,0
4,0x808327af2586c4bd:0x7d52d6ec742ab0e,478,271,0
...,...,...,...,...
171,0x80ea69de56c92db5:0x9c80bcccbaea80cb,218,147,162
172,0x80eaa58485538c21:0x38e0542d4d138714,56,37,0
173,0x80ec5eb7f04e6aff:0x2f0154760a7b6a31,93,67,13
174,0x80ece19d293cc681:0x6ade552bcafc3e26,76,49,0


In [43]:
# Save the .csv file
result_df.to_csv('/content/task1_124.csv', index = False)
print("task1_124.csv successfully generated!")

task1_124.csv successfully generated!


Some pre-processing on reviews column

In [44]:
# Convert the review into lower case
df['reviews'] = df['reviews'].apply(lambda x: x.lower() if isinstance(x, str) and x != "None" else x)

<ipython-input-44-0ed82666e08d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews'] = df['reviews'].apply(lambda x: x.lower() if isinstance(x, str) and x != "None" else x)


In [45]:
# Remove the emojis
def remove_emojis(text):
    # Define a regex pattern for emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )    # Reference [3]
    # Remove emojis from the text
    return emoji_pattern.sub(r'', text)

In [46]:
# Apply the function to the 'reviews' column
df.loc[:,'reviews'] = df['reviews'].apply(lambda x: remove_emojis(x))

**JSON file formatting**

In [47]:
# Create a if_pic column. Place Y if there's a picture, N otherwise
df['if_pic'] = df['pic'].apply(lambda x: 'Y' if isinstance(x, str) and 'url' in x else 'N')

<ipython-input-47-ab386d8282e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['if_pic'] = df['pic'].apply(lambda x: 'Y' if isinstance(x, str) and 'url' in x else 'N')


In [48]:
# Create a if_response column. Place Y if there's a response, N otherwise
df['if_response'] = df['response'].apply(lambda x: 'Y' if pd.notna(x) and x != 'None' else 'N')

<ipython-input-48-aca2c405f057>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['if_response'] = df['response'].apply(lambda x: 'Y' if pd.notna(x) and x != 'None' else 'N')


In [49]:
# In the output we have ratings as float, so need to type cast it into float
# Convert review_rating to float
df.loc[:,'ratings'] = df['ratings'].apply(lambda x: float(x) if x != 'None' else x)

In [50]:
# Process the data into the desired JSON format
json_output = {}

for gmap_id, group in df.groupby('gmapid'):
    reviews = []
    for _, row in group.iterrows():
        reviews.append({
            'user_id': row['user_id'],
            'time': row['time'],
            'review_rating': row['ratings'],
            'review_text': row['reviews'],
            'if_pic': row['if_pic'],
            'pic_dim': row['dimensions'],  # Convert the string representation back to list of tuples
            'if_response': row['if_response']
        })

    json_output[gmap_id] = {
        'reviews': reviews,
        'earliest_review_date': min(group['time']),
        'latest_review_date': max(group['time'])
    }

# Write the JSON output to a file
with open('/content/task1_124.json', 'w', encoding='utf-8') as f:
  json.dump(json_output, f, indent=3)

print("task1_124.json file successfully generated!")

task1_124.json file successfully generated!


## 9. References:


*   [1] https://www.geeksforgeeks.org/how-to-read-excel-multiple-sheets-in-python-pandas/
*   [2] https://pynative.com/python-regex-flags/
*   [3] https://gist.github.com/Alex-Just/e86110836f3f93fe7932290526529cd1









[Link to my workspace](https://colab.research.google.com/drive/1bghOQ5NYKWxSBQdVG22qmqm98hHHSmS3?usp=sharing)